In [1]:
import pmdarima as pm
import pandas as pd

stock_name = 'AAPL'
data_train = pd.read_csv('train_normalised.csv')
data_train.date = pd.to_datetime(data_train.date)
data_test = pd.read_csv('test_normalised.csv')
data_test.date = pd.to_datetime(data_test.date)

part_train =  len(data_train) / (len(data_train) + len(data_test))

df_series = pd.concat([data_train["close_" + stock_name], data_test["close_" + stock_name]])


trainwindow = 600
test = 150
ts = df_series.to_numpy()
# Fit the ARIMA model
model = pm.auto_arima(ts[:600], start_p=2, start_q=2, max_p=5, max_q=5, m=1, start_P=0, seasonal=False, d=None, D=None, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

# Display the model summary
print(model.summary())

# Get the optimal parameters
order = model.order
seasonal_order = model.seasonal_order

# Display the optimal parameters
print("Optimal ARIMA Order:", order)
print("Optimal Seasonal Order:", seasonal_order)

# Get the fitting coefficients
coefficients = model.params()

# Display the fitting coefficients
print("Fitting Coefficients:")
for i, coef in enumerate(coefficients):
    print(f"ARIMA({order[0]},{order[1]},{order[2]}) - Coefficient {i+1}: {coef}")

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-1300.355, Time=0.38 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-1303.317, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-1303.363, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-1303.198, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-1303.396, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-1304.312, Time=0.17 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-1302.344, Time=0.39 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-1302.343, Time=0.32 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-1302.329, Time=0.09 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-1302.443, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-1304.273, Time=0.12 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0] intercept
Total fit time: 1.752 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:       

In [7]:
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA


# Create the ARIMA model
numpred = len(data_test)
results = []
for d in range(1,2):
    for p in range(0,3):
        for q in range(0,3):

            predictions = []
            for i in tqdm(range(numpred,0,-1)):
                model = ARIMA(ts[-i - 600:-i], order=(p, d, q))

                model_fit = model.fit()

                pred = model_fit.forecast(step=1)
                predictions.append(pred[0])

            #mse
            print("For ({}, {}, {}), We have the MSE: {}".format(p,d,q,mean_squared_error(ts[-numpred:], predictions)))
            results.append([p,d,q,mean_squared_error(ts[-numpred:], predictions)])

  0%|          | 0/4613 [00:00<?, ?it/s]

/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['step']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
  1%|          | 41/4613 [00:01<02:27, 31.01it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 137/4613 [00:04<02:20, 31.77it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 158/4613 [00:05<02:28, 29.93it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: Converge

For (0, 1, 0), We have the MSE: 0.00923635174908962


 12%|█▏        | 537/4613 [00:20<03:29, 19.43it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 15%|█▍        | 680/4613 [00:26<02:55, 22.38it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 16%|█▋        | 760/4613 [00:30<02:38, 24.25it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
 17%|█▋        | 773/4613 [00:30<02:34, 24.78it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: Converg

For (0, 1, 1), We have the MSE: 0.00928532700982083


...

For (2, 1, 0), We have the MSE: 0.00929232458595765


  1%|          | 41/4613 [00:05<12:19,  6.19it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  2%|▏         | 74/4613 [00:10<12:43,  5.94it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 118/4613 [00:17<10:01,  7.47it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  3%|▎         | 158/4613 [00:23<13:02,  5.69it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: Convergen

For (2, 1, 1), We have the MSE: 0.009334230753247574


/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['step']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
  0%|          | 1/4613 [00:00<23:04,  3.33it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
  0%|          | 2/4613 [00:00<23:50,  3.22it/s]/Users/alex/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to

For (2, 1, 2), We have the MSE: 0.009389575075644288


In [8]:
resdf = pd.DataFrame(results, columns = ["p","d","q","mse"]).sort_values("mse")
resdf.sort_values("mse")


,p,d,q,mse
0,0,1,0,0.009236
1,0,1,1,0.009285
2,0,1,2,0.009291
4,1,1,1,0.009291
6,2,1,0,0.009292
5,1,1,2,0.009295
3,1,1,0,0.009296
7,2,1,1,0.009334
8,2,1,2,0.009390
